<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping Workflow

In this initial Jupyter workflow within PyCCAPT, we will guide you through the process of cropping atom probe data, whether it's originally collected using PyCCAPT or in various other formats such as EPOS, POS, ATO, and CSV. This workflow is designed to help you efficiently manage your atom probe data, focusing on both temporal and spatial cropping techniques. Additionally, we will cover essential calculations, including raw MC (Mass-to-Charge Ratio), pulses per ion, and ions per pulse. Lastly, you can explore how to save the cropped data in a range of formats, including PyCCAPT's native HDF5 format, EPOS, POS, ATO, and CSV, to suit your specific needs and preferences.
</div>

In [161]:
# Activate intractive functionality of matplotlib
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
from ipywidgets import fixed
from ipywidgets import interact_manual
from ipywidgets import widgets
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import share_variables
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools, tof_tools
from pyccapt.calibration.calibration_tools import mc_plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to crop. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [162]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, please install the pytables library by running the cell below. Befor running the cell, uncomment the cell by removing the # sign at the begining of the cell.
</div>


In [119]:
# check pytables library and install it
# try:
#     import tables
# except:
#     !conda install --yes --prefix {sys.prefix} pytables

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [163]:
# create object for selection of instrument specifications of the dataset
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flightPathLength_d, t0_d, max_mc)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=38.0, description='t0:')

FloatText(value=400.0, description='Max mc:')

In [164]:
# Calculate the maximum possible time of flight (TOF)
max_tof = int(tof_tools.mc2tof(max_mc.value, 1000, 0, 0, flightPathLength_d.value))
print('The maximum possible TOF is:', max_tof, 'ns')
print('=============================')
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode.value
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/' + variables.dataset_name  + '/load_crop/'
variables.result_data_name = variables.dataset_name 
variables.result_path = dataset_main_path + '/' + variables.dataset_name + '/load_crop/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)

print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')

# Create data farame out of hdf5 file dataset
dld_group_storage = data_tools.load_data(dataset_path, tdc.value, mode='raw')

# Remove the data with tof greater thatn Max TOF or below 0 ns
data = data_tools.remove_invalid_data(dld_group_storage, max_tof)
print('Total number of Ions:', len(data))

The maximum possible TOF is: 5010 ns
The data will be saved on the path: D:/pyccapt/tests/data/1704_Oct-13-2023_14-13_Al_Partick_B_test1/load_crop/
The dataset name after saving is: 1704_Oct-13-2023_14-13_Al_Partick_B_test1
The figures will be saved on the path: D:/pyccapt/tests/data/1704_Oct-13-2023_14-13_Al_Partick_B_test1/load_crop/
{'apt': ['high_voltage', 'hour', 'main_chamber_vacuum', 'minute', 'num_events', 'pulse', 'second', 'temperature', 'time_counter'], 'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'pulse', 'start_counter', 'time_data']}
The number of data over max_tof: 6
Total number of Ions: 86135


In [179]:
data

,high_voltage (V),pulse,start_counter,t (ns),x_det (cm),y_det (cm)
0,590.000000,328.000000,2053,2150.476807,0.682449,3.229388
1,590.000000,328.000000,8932,260.713715,2.569796,0.760816
2,590.000000,328.000000,9125,2527.927490,-1.361633,1.423674
3,600.000000,328.000000,11355,3693.067383,-3.836735,-0.101224
4,600.000000,328.000000,6645,4353.156738,-0.368980,2.680816
...,...,...,...,...,...,...
86082,5589.435059,1117.886963,17219,4440.013184,-3.069388,-0.502857
86083,5589.435059,1117.886963,17395,1589.869507,-1.534694,1.789388
86084,5589.435059,1117.886963,17556,844.926147,-1.746939,0.084898
86085,5589.435059,1117.886963,18277,684.874146,-0.724898,-1.335510


<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [180]:
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(False), data_crop=fixed(True), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_ini'));


interactive(children=(FloatText(value=5010.0, description='max_tof'), FloatText(value=1.0, description='frac')…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Boundaries of the selected(cropped) part of the graph is printed below
</div>

In [167]:
# Plot and selected experiment history
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(True), data_crop=fixed(False), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='hist_rect'));


interactive(children=(FloatText(value=5010.0, description='max_tof'), FloatText(value=1.0, description='frac')…

In [181]:
# Crop the dataset
print('Min Idx:', variables.selected_x1, 'Max Idx:', variables.selected_x2)
data_crop_t = data_loadcrop.crop_dataset(data, variables)

Min Idx: 2582.3869810215256 Max Idx: 83326.10599407126


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

# Spacial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [182]:
# Plot and select the FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
               draw_circle=fixed(False), data_crop=fixed(True), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_ini'));

interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

<b>The region selected in the previous step is displayed below.<b>
</div>

In [183]:
# plot selected area in FDM
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_t), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
                draw_circle=fixed(True), data_crop=fixed(False), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm_circle'));

interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

In [184]:
# Crop the dataset
print('center x:', variables.selected_x_fdm, 'center y:', variables.selected_y_fdm)
print('Radios:', variables.roi_fdm)
if variables.roi_fdm > 0:
    data_crop_spatial = data_loadcrop.crop_data_after_selection(data_crop_t, variables)
else:
    print('select the data spacialy from cell below')

center x: 0.1540211460611678 center y: -0.07049388791294531
Radios: 3.457007750049633


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

The final selected data after processing is shown below.
</div>

In [185]:
# Crop and plot the dataset
interact_manual(data_loadcrop.plot_crop_fdm, data=fixed(data_crop_spatial), variables=fixed(variables), frac=widgets.FloatText(value=1.0),
                bins=fixed((256,256)), figure_size=fixed((5,4)),
               draw_circle=fixed(False), data_crop=fixed(False), 
                save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='fdm'));

interactive(children=(FloatText(value=1.0, description='frac'), Dropdown(description='save', options=(('True',…

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Calculate pulses since the last event pulse and ions per pulse.
</div>

In [186]:
pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(data_crop_spatial)

# add two calculated array to the croped dataset
data_crop_spatial['pulse_pi'] = pulse_pi.astype(np.uintc)
data_crop_spatial['ion_pp'] = ion_pp.astype(np.uintc)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

The percentage of loss in ROI selection process.
</div>

In [187]:
# save the cropped data
print('tof Crop Loss {:.2f} %'.format((100 - (len(data_crop_spatial) / len(data)) * 100)))
#percentage of double event per pulse
print('percentage of double event per pulse', len(ion_pp[ion_pp != 1]) / float(len(ion_pp)))

tof Crop Loss 8.61 %
percentage of double event per pulse 0.213750063551782


In [153]:
# exctract needed data from Pandas data frame as an numpy array
variables.dld_high_voltage = data_crop_spatial['high_voltage (V)'].to_numpy()
variables.dld_pulse = data_crop_spatial['pulse'].to_numpy()
variables.dld_t = data_crop_spatial['t (ns)'].to_numpy()
variables.dld_x = data_crop_spatial['x_det (cm)'].to_numpy()
variables.dld_y = data_crop_spatial['y_det (cm)'].to_numpy()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub>. this correction would be helpful for the position of the peaks in the m/c calibration process.
</div>

In [154]:
def fine_tune_t_0(variables, t0_d, bin_size, log, mode, target, prominence, distance, percent, figname, lim):
    variables.mc = mc_tools.tof2mc(variables.dld_t, t0_d, variables.dld_high_voltage, variables.dld_x, variables.dld_y, flightPathLength_d.value, 
                                         variables.dld_pulse, mode=pulse_mode.value)
    if target == 'mc':
        mc_hist = mc_plot.AptHistPlotter(variables.mc[variables.mc < lim], variables)
        mc_hist.plot_histogram(bin_width=bin_size, mode=mode, label='mc', steps='stepfilled', log=log, fig_size=(9, 5))
    elif target == 'tof':
        mc_hist = mc_plot.AptHistPlotter(variables.dld_t[variables.dld_t < lim], variables)
        mc_hist.plot_histogram(bin_width=bin_size, mode=mode, label='tof', steps='stepfilled', log=log, fig_size=(9, 5))
    
    if mode != 'normalized':
        mc_hist.find_peaks_and_widths(prominence=prominence, distance=distance, percent=percent)
        mc_hist.plot_peaks()
        mc_hist.plot_hist_info_legend(label='mc', bin=0.1, background=None, loc='right')
    
    mc_hist.save_fig(label=mode, fig_name=figname)

interact_manual(fine_tune_t_0, variables=fixed(variables), t0_d=widgets.FloatText(value=t0_d.value), bin_size=widgets.FloatText(value=0.1), 
                log=widgets.Dropdown(options=[('True', True), ('False', False)]), mode=widgets.Dropdown(options=[('normal', 'normal'), ('normalized', 'normalized')]),
                target=widgets.Dropdown(options=[('mc', 'mc'), ('tof', 'tof')]), prominence=widgets.IntText(value=10), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), figname=widgets.Text(value='hist'));

interactive(children=(FloatText(value=38.0, description='t0_d'), FloatText(value=0.1, description='bin_size'),…

In [155]:
data_crop_spatial_back = data_crop_spatial.copy()

In [156]:
data_crop_spatial_back.insert(0, 'x (nm)', np.zeros(len(variables.dld_t)))
data_crop_spatial_back.insert(1, 'y (nm)', np.zeros(len(variables.dld_t)))
data_crop_spatial_back.insert(2,'z (nm)', np.zeros(len(variables.dld_t)))
data_crop_spatial_back.insert(3,'mc_c (Da)', np.zeros(len(variables.dld_t)))
data_crop_spatial_back.insert(4, 'mc (Da)', variables.mc)
data_crop_spatial_back.insert(8,'t_c (ns)', np.zeros(len(variables.dld_t)))

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Remove the data with m/c greater than max m/c and x, y, t = 0
</div>

In [157]:
# Remove the data with mc biger than max mc
mask = (data_crop_spatial_back['mc (Da)'].to_numpy() > max_mc.value)
print('The number of data over max_mc:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

# Remove the data with x,y,t = 0
mask1 = (data_crop_spatial_back['x (nm)'].to_numpy() == 0)
mask2 = (data_crop_spatial_back['y (nm)'].to_numpy() == 0)
mask3 = (data_crop_spatial_back['t (ns)'].to_numpy() == 0)
mask = np.logical_and(mask1, mask2)
mask = np.logical_and(mask, mask3)
print('The number of data with having t, x, and y equal to zero is:', len(mask[mask==True]))
data_crop_spatial_back.drop(np.where(mask)[0], inplace=True)
data_crop_spatial_back.reset_index(inplace=True, drop=True)

The number of data over max_mc: 16035
The number of data with having t, x, and y equal to zero is: 0


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

The final cropped dataset is displayed below.
</div>

In [158]:
data_crop_spatial_back

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,0.0,31.379771,4580.72,687.108,0,0.0,667.797750,0.218776,2.034286,0,0
1,0.0,0.0,0.0,0.0,29.556835,4580.72,687.108,0,0.0,648.780516,-1.998367,-0.091429,0,0
2,0.0,0.0,0.0,0.0,30.162143,4580.72,687.108,0,0.0,651.955770,-1.662041,-0.088163,0,0
3,0.0,0.0,0.0,0.0,30.463507,4580.72,687.108,0,0.0,651.921480,-0.976327,-0.760816,0,0
4,0.0,0.0,0.0,0.0,32.030306,4580.72,687.108,0,0.0,675.574722,-2.128980,0.398367,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594715,0.0,0.0,0.0,0.0,31.218738,5121.26,768.189,0,0.0,631.642374,1.857959,0.734694,0,0
594716,0.0,0.0,0.0,0.0,30.473116,5121.26,768.189,0,0.0,640.242306,2.922449,1.498776,0,0
594717,0.0,0.0,0.0,0.0,31.519006,5121.26,768.189,0,0.0,630.222768,0.313469,1.453061,0,0
594718,0.0,0.0,0.0,0.0,31.656391,5121.26,768.189,0,0.0,628.309386,-0.927347,0.153469,0,0


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

The data types of the final cropped dataset is displayed below.
</div>

In [159]:
data_crop_spatial_back.dtypes

x (nm)              float64
y (nm)              float64
z (nm)              float64
mc_c (Da)           float64
mc (Da)             float64
high_voltage (V)    float64
pulse               float64
start_counter        uint32
t_c (ns)            float64
t (ns)              float64
x_det (cm)          float64
y_det (cm)          float64
pulse_pi             uint32
ion_pp               uint32
dtype: object

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the cropped dataset. You can specify te output format from list below. The output formats are HDF5, EPOS, POS, ATO, and CSV. The output file will be saved in the same directory as the original dataset file in a new directory nammed load_crop.
</div>

In [160]:
interact_manual(data_tools.save_data, data=fixed(data_crop_spatial_back), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…